# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [68]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
#     TODO: Implement Function
#     print("source_text:\n{}".format(source_text))
#     print()
#     print("source_vocab:\n{}".format(source_vocab_to_int))

    source_id_text = [[source_vocab_to_int.get(word, source_vocab_to_int['<UNK>']) for word in line.split(" ")] for line in source_text.split('\n')]
#     print()
#     print('source_id_text:\n{}'.format(source_id_text))
    
#     print()
#     print("target_text:\n{}".format(target_text))
#     print()
#     print("target_vocab_to_int:\n{}".format(target_vocab_to_int))
    
    target_id_text = [[target_vocab_to_int.get(word, target_vocab_to_int['<UNK>']) for word in line.split(" ")] + [target_vocab_to_int['<EOS>']] for line in target_text.split('\n')]
#     print()
#     print('target_id_text:\n{}'.format(target_id_text))

    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [69]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [3]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [5]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    target_sequence_length = tf.placeholder(tf.int32, (None, ), name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_legth = tf.placeholder(tf.int32, (None,), name='source_sequence_length')
    return input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_legth


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [6]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)
    return dec_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [7]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    # Encoder embedding
    enc_embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    #RNN cell
    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size, 
                                          initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        drop = tf.contrib.rnn.DropoutWrapper(enc_cell, output_keep_prob=keep_prob)
        return drop
    
    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    
    enc_output, enc_state = tf.nn.dynamic_rnn(enc_cell, enc_embed_input, sequence_length=source_sequence_length, dtype=tf.float32)
    return enc_output, enc_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [8]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    # Helper for the training process. Used by BasicDecoder to read inputs
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input, 
                                                        sequence_length=target_sequence_length,
                                                        time_major=False)

    # Basic Decoder
    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                                      training_helper,
                                                      encoder_state, 
                                                      output_layer)

    # Perform dynamic decoding using the decoder
    training_decoder_output = tf.contrib.seq2seq.dynamic_decode(training_decoder, 
                                                               impute_finished=True,
                                                                   maximum_iterations=max_summary_length)[0]
    return training_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [9]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    # Reuses the same parameters trained by the training process
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32),
                           [batch_size], 
                           name='start_tokens')

    # Helper for the inference process.
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, 
                                                                start_tokens, end_of_sequence_id)

    # Basic decoder
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, 
                                                       inference_helper,
                                                       encoder_state, 
                                                       output_layer)

    # Perform dynamic decoding using the decoder
    inference_decoder_output = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                                impute_finished=True,
                                                                maximum_iterations=max_target_sequence_length)[0]
    return inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [10]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # Decoder Embedding
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    # Construct the decoder Cell
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size, 
                                          initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        drop = tf.contrib.rnn.DropoutWrapper(dec_cell, output_keep_prob=keep_prob)
        return drop
    
    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    # Dense layer to translate the decoder's output at each time 
    # step into a choice from the target vocabulary
    output_layer = Dense(target_vocab_size, 
                        kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    # Set up a training decoder and an inference decoder
    # Training Decoder
    with tf.variable_scope("decode"):
        training_decoder_output = decoding_layer_train(encoder_state, 
                                                       dec_cell, 
                                                       dec_embed_input, 
                                                       target_sequence_length, 
                                                       max_target_sequence_length,
                                                       output_layer, keep_prob)
    
    # Inference Decoder
    # Reuse the same parameters trained by the training process
    with tf.variable_scope("decode", reuse=True):
        inference_decoder_output = decoding_layer_infer(encoder_state,
                                                        dec_cell, 
                                                        dec_embeddings, 
                                                        target_vocab_to_int['<GO>'], 
                                                        target_vocab_to_int['<EOS>'],
                                                        max_target_sequence_length,
                                                        target_vocab_size, 
                                                        output_layer, 
                                                        batch_size, 
                                                        keep_prob)
        
    return training_decoder_output, inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [12]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # Pass the input data through the encoder. We'll ignore the encoder output, but use the state
    _, enc_state = encoding_layer(input_data, 
                                  rnn_size, 
                                  num_layers,
                                  keep_prob,  
                                  source_sequence_length, 
                                  source_vocab_size,
                                  enc_embedding_size)
    
    # Prepare the target sequences we'll feed to the decoder in training mode
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    # Pass encoder state and decoder inputs to the decoders
    training_decoder_output, inference_decoder_output = decoding_layer(dec_input, 
                                                                       enc_state,
                                                                       target_sequence_length,
                                                                       max_target_sentence_length,
                                                                       rnn_size,
                                                                       num_layers,
                                                                       target_vocab_to_int,
                                                                       target_vocab_size,
                                                                       batch_size,
                                                                       keep_prob,
                                                                       dec_embedding_size)
    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [13]:
# Number of Epochs
epochs = 60
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 50
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 15
decoding_embedding_size = 15
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.7
display_step = 20

### Build the Graph
Build the graph using the neural network you implemented.

In [14]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch   20/1077 - Train Accuracy: 0.2316, Validation Accuracy: 0.3050, Loss: 4.7285
Epoch   0 Batch   40/1077 - Train Accuracy: 0.2648, Validation Accuracy: 0.3356, Loss: 3.7521
Epoch   0 Batch   60/1077 - Train Accuracy: 0.2987, Validation Accuracy: 0.3366, Loss: 3.4242
Epoch   0 Batch   80/1077 - Train Accuracy: 0.2719, Validation Accuracy: 0.3363, Loss: 3.3617
Epoch   0 Batch  100/1077 - Train Accuracy: 0.2617, Validation Accuracy: 0.3363, Loss: 3.3068
Epoch   0 Batch  120/1077 - Train Accuracy: 0.3000, Validation Accuracy: 0.3640, Loss: 3.2172
Epoch   0 Batch  140/1077 - Train Accuracy: 0.2595, Validation Accuracy: 0.3583, Loss: 3.3036
Epoch   0 Batch  160/1077 - Train Accuracy: 0.3031, Validation Accuracy: 0.3594, Loss: 3.0430
Epoch   0 Batch  180/1077 - Train Accuracy: 0.3172, Validation Accuracy: 0.3718, Loss: 3.0146
Epoch   0 Batch  200/1077 - Train Accuracy: 0.3367, Validation Accuracy: 0.3952, Loss: 2.9541
Epoch   0 Batch  220/1077 - Train Accuracy: 0.3265, Validati

Epoch   1 Batch  720/1077 - Train Accuracy: 0.4433, Validation Accuracy: 0.5281, Loss: 1.1853
Epoch   1 Batch  740/1077 - Train Accuracy: 0.5098, Validation Accuracy: 0.5312, Loss: 1.0818
Epoch   1 Batch  760/1077 - Train Accuracy: 0.4926, Validation Accuracy: 0.5405, Loss: 1.1317
Epoch   1 Batch  780/1077 - Train Accuracy: 0.4746, Validation Accuracy: 0.5234, Loss: 1.1296
Epoch   1 Batch  800/1077 - Train Accuracy: 0.4559, Validation Accuracy: 0.5284, Loss: 1.1074
Epoch   1 Batch  820/1077 - Train Accuracy: 0.5055, Validation Accuracy: 0.5518, Loss: 1.0918
Epoch   1 Batch  840/1077 - Train Accuracy: 0.4867, Validation Accuracy: 0.5302, Loss: 1.0686
Epoch   1 Batch  860/1077 - Train Accuracy: 0.4952, Validation Accuracy: 0.5266, Loss: 1.0430
Epoch   1 Batch  880/1077 - Train Accuracy: 0.5223, Validation Accuracy: 0.5494, Loss: 1.0253
Epoch   1 Batch  900/1077 - Train Accuracy: 0.4898, Validation Accuracy: 0.5376, Loss: 1.0954
Epoch   1 Batch  920/1077 - Train Accuracy: 0.4773, Validati

Epoch   3 Batch  360/1077 - Train Accuracy: 0.5910, Validation Accuracy: 0.6115, Loss: 0.7802
Epoch   3 Batch  380/1077 - Train Accuracy: 0.5961, Validation Accuracy: 0.6023, Loss: 0.7489
Epoch   3 Batch  400/1077 - Train Accuracy: 0.5973, Validation Accuracy: 0.5969, Loss: 0.7718
Epoch   3 Batch  420/1077 - Train Accuracy: 0.5559, Validation Accuracy: 0.6172, Loss: 0.7465
Epoch   3 Batch  440/1077 - Train Accuracy: 0.5801, Validation Accuracy: 0.5969, Loss: 0.7572
Epoch   3 Batch  460/1077 - Train Accuracy: 0.5492, Validation Accuracy: 0.5973, Loss: 0.7767
Epoch   3 Batch  480/1077 - Train Accuracy: 0.5851, Validation Accuracy: 0.6001, Loss: 0.7787
Epoch   3 Batch  500/1077 - Train Accuracy: 0.5680, Validation Accuracy: 0.6069, Loss: 0.7428
Epoch   3 Batch  520/1077 - Train Accuracy: 0.6157, Validation Accuracy: 0.6104, Loss: 0.7049
Epoch   3 Batch  540/1077 - Train Accuracy: 0.5699, Validation Accuracy: 0.6048, Loss: 0.7135
Epoch   3 Batch  560/1077 - Train Accuracy: 0.5785, Validati

Epoch   4 Batch 1060/1077 - Train Accuracy: 0.6312, Validation Accuracy: 0.6456, Loss: 0.5969
Epoch   5 Batch   20/1077 - Train Accuracy: 0.5793, Validation Accuracy: 0.6442, Loss: 0.5890
Epoch   5 Batch   40/1077 - Train Accuracy: 0.6051, Validation Accuracy: 0.6410, Loss: 0.6166
Epoch   5 Batch   60/1077 - Train Accuracy: 0.6209, Validation Accuracy: 0.6360, Loss: 0.6080
Epoch   5 Batch   80/1077 - Train Accuracy: 0.6215, Validation Accuracy: 0.6413, Loss: 0.6247
Epoch   5 Batch  100/1077 - Train Accuracy: 0.6383, Validation Accuracy: 0.6516, Loss: 0.6307
Epoch   5 Batch  120/1077 - Train Accuracy: 0.6359, Validation Accuracy: 0.6424, Loss: 0.6206
Epoch   5 Batch  140/1077 - Train Accuracy: 0.5917, Validation Accuracy: 0.6438, Loss: 0.6334
Epoch   5 Batch  160/1077 - Train Accuracy: 0.6613, Validation Accuracy: 0.6424, Loss: 0.5848
Epoch   5 Batch  180/1077 - Train Accuracy: 0.6195, Validation Accuracy: 0.6367, Loss: 0.5959
Epoch   5 Batch  200/1077 - Train Accuracy: 0.6043, Validati

Epoch   6 Batch  700/1077 - Train Accuracy: 0.6629, Validation Accuracy: 0.6903, Loss: 0.5148
Epoch   6 Batch  720/1077 - Train Accuracy: 0.6591, Validation Accuracy: 0.6875, Loss: 0.5585
Epoch   6 Batch  740/1077 - Train Accuracy: 0.6715, Validation Accuracy: 0.6761, Loss: 0.4969
Epoch   6 Batch  760/1077 - Train Accuracy: 0.6813, Validation Accuracy: 0.6999, Loss: 0.5253
Epoch   6 Batch  780/1077 - Train Accuracy: 0.6891, Validation Accuracy: 0.6751, Loss: 0.5307
Epoch   6 Batch  800/1077 - Train Accuracy: 0.6855, Validation Accuracy: 0.6815, Loss: 0.5003
Epoch   6 Batch  820/1077 - Train Accuracy: 0.6582, Validation Accuracy: 0.6967, Loss: 0.5242
Epoch   6 Batch  840/1077 - Train Accuracy: 0.7047, Validation Accuracy: 0.6999, Loss: 0.4853
Epoch   6 Batch  860/1077 - Train Accuracy: 0.6860, Validation Accuracy: 0.6779, Loss: 0.5080
Epoch   6 Batch  880/1077 - Train Accuracy: 0.7273, Validation Accuracy: 0.7038, Loss: 0.4815
Epoch   6 Batch  900/1077 - Train Accuracy: 0.6988, Validati

...

Epoch  53 Batch  160/1077 - Train Accuracy: 0.9672, Validation Accuracy: 0.9407, Loss: 0.0610
Epoch  53 Batch  180/1077 - Train Accuracy: 0.9582, Validation Accuracy: 0.9442, Loss: 0.0547
Epoch  53 Batch  200/1077 - Train Accuracy: 0.9461, Validation Accuracy: 0.9553, Loss: 0.0749
Epoch  53 Batch  220/1077 - Train Accuracy: 0.9688, Validation Accuracy: 0.9453, Loss: 0.0820
Epoch  53 Batch  240/1077 - Train Accuracy: 0.9691, Validation Accuracy: 0.9492, Loss: 0.0723
Epoch  53 Batch  260/1077 - Train Accuracy: 0.9368, Validation Accuracy: 0.9517, Loss: 0.0672
Epoch  53 Batch  280/1077 - Train Accuracy: 0.9277, Validation Accuracy: 0.9524, Loss: 0.0848
Epoch  53 Batch  300/1077 - Train Accuracy: 0.9613, Validation Accuracy: 0.9457, Loss: 0.0627
Epoch  53 Batch  320/1077 - Train Accuracy: 0.9520, Validation Accuracy: 0.9542, Loss: 0.0856
Epoch  53 Batch  340/1077 - Train Accuracy: 0.9581, Validation Accuracy: 0.9482, Loss: 0.0738
Epoch  53 Batch  360/1077 - Train Accuracy: 0.9816, Validati

Epoch  54 Batch  860/1077 - Train Accuracy: 0.9453, Validation Accuracy: 0.9524, Loss: 0.0862
Epoch  54 Batch  880/1077 - Train Accuracy: 0.9527, Validation Accuracy: 0.9652, Loss: 0.0631
Epoch  54 Batch  900/1077 - Train Accuracy: 0.9449, Validation Accuracy: 0.9489, Loss: 0.0785
Epoch  54 Batch  920/1077 - Train Accuracy: 0.9566, Validation Accuracy: 0.9439, Loss: 0.0700
Epoch  54 Batch  940/1077 - Train Accuracy: 0.9473, Validation Accuracy: 0.9556, Loss: 0.0614
Epoch  54 Batch  960/1077 - Train Accuracy: 0.9449, Validation Accuracy: 0.9485, Loss: 0.0796
Epoch  54 Batch  980/1077 - Train Accuracy: 0.9414, Validation Accuracy: 0.9513, Loss: 0.0781
Epoch  54 Batch 1000/1077 - Train Accuracy: 0.9245, Validation Accuracy: 0.9510, Loss: 0.0689
Epoch  54 Batch 1020/1077 - Train Accuracy: 0.9703, Validation Accuracy: 0.9329, Loss: 0.0708
Epoch  54 Batch 1040/1077 - Train Accuracy: 0.9704, Validation Accuracy: 0.9592, Loss: 0.0626
Epoch  54 Batch 1060/1077 - Train Accuracy: 0.9379, Validati

Epoch  56 Batch  500/1077 - Train Accuracy: 0.9563, Validation Accuracy: 0.9599, Loss: 0.0629
Epoch  56 Batch  520/1077 - Train Accuracy: 0.9699, Validation Accuracy: 0.9354, Loss: 0.0624
Epoch  56 Batch  540/1077 - Train Accuracy: 0.9688, Validation Accuracy: 0.9549, Loss: 0.0636
Epoch  56 Batch  560/1077 - Train Accuracy: 0.9316, Validation Accuracy: 0.9446, Loss: 0.0682
Epoch  56 Batch  580/1077 - Train Accuracy: 0.9557, Validation Accuracy: 0.9503, Loss: 0.0599
Epoch  56 Batch  600/1077 - Train Accuracy: 0.9483, Validation Accuracy: 0.9510, Loss: 0.0771
Epoch  56 Batch  620/1077 - Train Accuracy: 0.9719, Validation Accuracy: 0.9389, Loss: 0.0653
Epoch  56 Batch  640/1077 - Train Accuracy: 0.9621, Validation Accuracy: 0.9592, Loss: 0.0661
Epoch  56 Batch  660/1077 - Train Accuracy: 0.9625, Validation Accuracy: 0.9656, Loss: 0.0684
Epoch  56 Batch  680/1077 - Train Accuracy: 0.9308, Validation Accuracy: 0.9545, Loss: 0.0796
Epoch  56 Batch  700/1077 - Train Accuracy: 0.9691, Validati

Epoch  58 Batch  140/1077 - Train Accuracy: 0.9441, Validation Accuracy: 0.9492, Loss: 0.0635
Epoch  58 Batch  160/1077 - Train Accuracy: 0.9691, Validation Accuracy: 0.9446, Loss: 0.0570
Epoch  58 Batch  180/1077 - Train Accuracy: 0.9637, Validation Accuracy: 0.9556, Loss: 0.0683
Epoch  58 Batch  200/1077 - Train Accuracy: 0.9641, Validation Accuracy: 0.9606, Loss: 0.0700
Epoch  58 Batch  220/1077 - Train Accuracy: 0.9605, Validation Accuracy: 0.9556, Loss: 0.0720
Epoch  58 Batch  240/1077 - Train Accuracy: 0.9656, Validation Accuracy: 0.9506, Loss: 0.0636
Epoch  58 Batch  260/1077 - Train Accuracy: 0.9501, Validation Accuracy: 0.9538, Loss: 0.0532
Epoch  58 Batch  280/1077 - Train Accuracy: 0.9207, Validation Accuracy: 0.9535, Loss: 0.0801
Epoch  58 Batch  300/1077 - Train Accuracy: 0.9634, Validation Accuracy: 0.9439, Loss: 0.0645
Epoch  58 Batch  320/1077 - Train Accuracy: 0.9566, Validation Accuracy: 0.9450, Loss: 0.0890
Epoch  58 Batch  340/1077 - Train Accuracy: 0.9741, Validati

Epoch  59 Batch  840/1077 - Train Accuracy: 0.9723, Validation Accuracy: 0.9570, Loss: 0.0603
Epoch  59 Batch  860/1077 - Train Accuracy: 0.9431, Validation Accuracy: 0.9528, Loss: 0.0698
Epoch  59 Batch  880/1077 - Train Accuracy: 0.9539, Validation Accuracy: 0.9503, Loss: 0.0630
Epoch  59 Batch  900/1077 - Train Accuracy: 0.9633, Validation Accuracy: 0.9577, Loss: 0.0662
Epoch  59 Batch  920/1077 - Train Accuracy: 0.9598, Validation Accuracy: 0.9556, Loss: 0.0651
Epoch  59 Batch  940/1077 - Train Accuracy: 0.9473, Validation Accuracy: 0.9595, Loss: 0.0695
Epoch  59 Batch  960/1077 - Train Accuracy: 0.9613, Validation Accuracy: 0.9545, Loss: 0.0723
Epoch  59 Batch  980/1077 - Train Accuracy: 0.9402, Validation Accuracy: 0.9535, Loss: 0.0785
Epoch  59 Batch 1000/1077 - Train Accuracy: 0.9442, Validation Accuracy: 0.9556, Loss: 0.0669
Epoch  59 Batch 1020/1077 - Train Accuracy: 0.9676, Validation Accuracy: 0.9581, Loss: 0.0770
Epoch  59 Batch 1040/1077 - Train Accuracy: 0.9799, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [38]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
- Convert words not in the vocabulary, to the `<UNK>` word id.

In [39]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    sentence_lowercase = sentence.lower()
    sent = [word for word in sentence_lowercase.split()]
    sequence_length = 7
    prepr_sent = [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in sentence_lowercase.split()] + [vocab_to_int['<PAD>']]*(sequence_length-len(sentence_lowercase))
    return prepr_sent


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [40]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [56, 161, 107, 7, 197, 21, 124]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [109, 246, 55, 316, 249, 1]
  French Words: l' anglais merveilleux hiver . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.